In [1]:
import torch
import pandas as pd
import numpy as np 
import matplotlib.pylab as plt 

# our scripts and functions
from src.gp.gaussianprocess import GaussianProcess
import src.cosmofuncs as cf
import src.model as sm
import src.compression as sc 
import src.optimisation as so
import utils.helpers as hp
import setting as st


plt.rc('text', usetex=True)
plt.rc('font',**{'family':'sans-serif','serif':['Palatino']})
figSize  = (12, 8)
fontSize = 20

In [2]:
model = sm.AppMag(load_data=True)

# Compression

In [3]:
comp = sc.moped(load_data = True)
moped_vectors = comp.vectors(model.mle)

0 0 :  1.0000
1 0 : -0.0000
1 1 :  1.0000
2 0 :  0.0000
2 1 :  0.0000
2 2 :  1.0000
3 0 : -0.0000
3 1 : -0.0000
3 2 : -0.0000
3 3 :  1.0000
4 0 :  0.0000
4 1 : -0.0000
4 2 : -0.0000
4 3 :  0.0000
4 4 :  1.0000
5 0 :  0.0000
5 1 : -0.0000
5 2 : -0.0000
5 3 : -0.0000
5 4 : -0.0000
5 5 :  1.0000


# Emulation

We will emulate the MOPED coefficients. 

In [4]:
num = 500
csv = pd.read_csv('simulations/simulations_'+str(num)+'.csv')
mopedcoeffs = csv[[f'exact_{i+1}' for i in range(6)]]

In [5]:
# these are the cosmological and nuisance parameters
inputs = torch.from_numpy(csv.iloc[:,0:6].values)

niter = 500
lr = 0.01
nrestart = 3 

In [6]:
gp_moped = {}
for i in range(6):
    # this is the i^th MOPED coefficient
    output = torch.from_numpy(mopedcoeffs.iloc[:,i].values)
    
    # train the GP
    GP_module = GaussianProcess(inputs, output, 1E-5)
    start = torch.randn((inputs.shape[1]+1,))
    optimum = GP_module.optimisation(parameters = start, niter = niter, l_rate=lr, nrestart = nrestart)
    gp_moped[f'gp_{i+1}'] = GP_module

# Test Prediction

In [11]:
param = torch.tensor([0.15, -1.0, -19.04, -0.04, 0.125, 2.60])

In [12]:
theory = model.theory_only(param)

In [13]:
comp.compute_coefficients(theory)

tensor([-1.1648e+03, -5.4407e+02,  6.9902e+02,  8.8756e-01,  1.4377e+01,
         3.2957e+01])

In [15]:
torch.cat([gp_moped[f'gp_{i+1}'].mean_prediction(param).data for i in range(6)])

tensor([-1.1648e+03, -5.4408e+02,  6.9902e+02,  8.9219e-01,  1.4376e+01,
         3.2950e+01], dtype=torch.float64)

# Accuracy Test

- Sample a few points from the prior. 
- Compute the MOPED coefficients. 
- Calculate the accuracy for each emulator.

$$
A = \dfrac{\mu - \mu_{\textrm{emu}}}{\mu} \times 100
$$

In [36]:
import src.trainingpoints as stp

In [37]:
module = stp.simulations(load_data = True, nlhs = 500)

In [50]:
ntest = 100

In [56]:
samples_om = module.dist['p0'].rsample(torch.Size([ntest])).numpy().reshape(ntest, 1)
samples_w0 = module.dist['p1'].rsample(torch.Size([ntest])).numpy().reshape(ntest, 1)
samples_nuisance = module.dist['p_nuisance'].rsample(torch.Size([ntest])).numpy().reshape(ntest, 4)

In [58]:
testpoints = np.concatenate([samples_om, samples_w0, samples_nuisance], axis = 1)

To continue here.

# MCMC

To couple emulator with MCMC routine. 